In [3]:
import os

In [4]:
os.environ["OPENAI_API_VERSION"] = "2023-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = ""
os.environ["AZURE_OPENAI_API_KEY"] = ""

In [5]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
model_40_temp_08 = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4-turbo-2024-04-09",
    temperature=0.8,
)

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}"),
    ("user", "{input}")
])

chain = prompt | model_40_temp_08 | output_parser

In [7]:
system = """
    You are world class mathematician.
"""

prompt_text = """
    I need to compute 23 * 3 - 89
"""

print("System: ", system)
print("************************************************")
resp = chain.invoke({ "system": system, "input": prompt_text })
resp

System:  
    You are world class mathematician.

************************************************


'To solve the expression \\(23 \\times 3 - 89\\):\n\nFirst, perform the multiplication:\n\\(23 \\times 3 = 69\\)\n\nNext, subtract 89 from the result:\n\\(69 - 89 = -20\\)\n\nSo, \\(23 \\times 3 - 89 = -20\\).'

In [8]:
from langchain.agents import tool


@tool
def multiply(a: int, b:int) -> int:
    """Multiplies two numbers"""
    return a*b

@tool
def substract(a: int, b:int) -> int:
    """Return the difference between two numbers, return a-b"""
    return a-b

tools = [multiply, substract]

llm_with_tools = model_40_temp_08.bind_tools(tools)

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "{system}"),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


agent = (
        {
            "input": lambda x: x["input"],
            "system": lambda x: x["system"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
            "chat_history": lambda x: x["chat_history"],
        }
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
)

from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [12]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
input1 = "What is 2 * 6 ?"
result = agent_executor.invoke({"input": input1, "system": "You are world class mathematician", "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "And what is that multiplied by 10 ?", "system": "You are world class mathematician", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 2, 'b': 6}`


12The result of \(2 \times 6\) is \(12\).

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 12, 'b': 10}`


120The result of \(12 \times 10\) is \(120\).

> Finished chain.


{'input': 'And what is that multiplied by 10 ?',
 'system': 'You are world class mathematician',
 'chat_history': [HumanMessage(content='What is 2 * 6 ?'),
  AIMessage(content='The result of \\(2 \\times 6\\) is \\(12\\).')],
 'output': 'The result of \\(12 \\times 10\\) is \\(120\\).'}